<a href="https://colab.research.google.com/github/CansecoSergio/python-data-science/blob/main/M5%20Sergio%20Canseco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Reto Módulo 5 Sergio Canseco**

**1.- Instalación de streamlit, ngrok y creación del túnel de comunicación para visualizar la aplicación.**

In [1]:
# instalar streamlit
!pip install -q streamlit

# bajar ngrok
!wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
!tar xvf ngrok-v3-stable-linux-amd64.tgz

# guardar token
!./ngrok config add-authtoken 2fnTvA033DsLeciIE8BLJyiZ2It_4cpbKgtpPgRHxSQ7sfhWM

# levantar tunel
!./ngrok http 8501 &>/content/ngrok.log &

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.8 MB/s eta 0:00:00
--2024-05-09 16:58:20--  https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 54.161.241.46, 54.237.133.81, 18.205.222.128, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.161.241.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8949853 (8.5M) [application/octet-stream]
Saving to: ‘ngrok-v3-stable-linux-amd64.tgz’

ngrok-v3-stable-lin 100%[===================>]   8.54M  28.3MB/s    in 0.3s    

2024-05-09 16:58:21 (28.3 MB/s) - ‘ngrok-v3-stable-linux-amd64.tgz’ saved [8949853/8949853]

ngrok
Authtoke

**2.- Creación del archivo py que será ejecutado por streamlit**

In [126]:
%%writefile employees.py

import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_palette("PuBuGn_d")
st.set_page_config(layout="wide")
st.title("Tablero del Hackathon HackerEarth 2020")
st.write("El objetivo de este sitio es poder generar información explicativa del fenómeno de deserción laboral así como también la afectación a las empresas y organizaciones.")

@st.cache_data(ttl="1d")
def load_info_employees(): #n_rows=500 parámetro para desarrollo, solo traer por default 500
  data = pd.read_csv("/content/drive/MyDrive/Archivos Collab/M5/Employees.csv")
  return data

sidebar = st.sidebar
sidebar.title("Filtros")

full_employees_data = load_info_employees()
employees_data = load_info_employees()

employee_id = sidebar.text_input("Employee ID", placeholder="Ej. EID_9999")
education_levels = sidebar.multiselect("Education Levels: ", sorted(employees_data['Education_Level'].unique()))
cities = sidebar.multiselect("Cities: ", sorted(employees_data['Hometown'].unique()))
units = sidebar.multiselect("Units: ", sorted(employees_data['Unit'].unique()))
full_data = sidebar.checkbox("Mostrar dataframe original")

if employee_id:
  employees_data = employees_data[employees_data['Employee_ID'].str.contains(employee_id, case=False)]

if education_levels:
  employees_data = employees_data[employees_data['Education_Level'].isin(education_levels)]

if cities:
  employees_data = employees_data[employees_data['Hometown'].isin(cities)]

if units:
  employees_data = employees_data[employees_data['Unit'].isin(units)]

#Visualizacion del dataframe completo
if full_data:
  st.divider()
  st.header("Dataframe original")
  st.dataframe(full_employees_data, use_container_width=True)
  st.write(f"Total de registros: {full_employees_data.shape[0]}")

st.divider()

st.header("Resultados")
st.dataframe(employees_data, use_container_width=True)
st.write(f"Total de registros: {employees_data.shape[0]}")

st.divider()

col1, col2 = st.columns(2)
with col1:
  fig_1 = plt.figure(figsize=(6, 6))
  sns.histplot(employees_data['Age'], bins=10, color='#6AAE3D', edgecolor='#27272A').set_title("Histograma por Edad")
  st.pyplot(fig_1)

with col2:
  fig_2 = plt.figure(figsize=(6, 6))
  sns.countplot(employees_data['Unit'], palette=['#0078D4',"#35B7EC"]).set_title("Frecuencia de Empleados por Unidad")
  st.pyplot(fig_2)

col3, col4 = st.columns(2)
with col3:
  fig_3 = plt.figure(figsize=(6, 6))
  data_mean = employees_data.groupby(by="Hometown").mean(numeric_only=True)
  data_mean.reset_index()
  sns.lineplot(data=data_mean, x="Hometown", y="Attrition_rate", marker="o").set_title("Índice de Deserción por Ciudad")
  st.pyplot(fig_3)

with col4:
  fig_4 = plt.figure(figsize=(6, 6))
  sns.scatterplot(x=employees_data['Age'],y=employees_data['Attrition_rate'], color="#FF6954").set_title("Correlación Edad y Tasa de Deserción")
  st.pyplot(fig_4)

col5, col6 = st.columns(2)
with col5:
  fig_5 = plt.figure(figsize=(6, 6))
  sns.scatterplot(x=employees_data['Time_of_service'],y=employees_data['Attrition_rate'], color="#FFA421").set_title("Correlación Tiempo de Servicio y Tasa de Deserción")
  st.pyplot(fig_5)





Overwriting employees.py


**3.- Ejecución de streamlit**

In [3]:
!streamlit run /content/employees.py &>/content/streamlit.log &

In [4]:
!curl -s http://localhost:4040/api/tunnels | python3 -c 'import sys, json; print("Execute the next cell and then go to the following URL: " +json.load(sys.stdin)["tunnels"][0]["public_url"])'

Execute the next cell and then go to the following URL: https://91eb-34-68-9-29.ngrok-free.app


**4.- Análisis de la información**

In [15]:
import pandas as pd

data = pd.read_csv("/content/drive/MyDrive/Archivos Collab/M5/Employees.csv")
data.reset_index(drop=True, inplace=True)
data

,Employee_ID,Gender,Age,Education_Level,Relationship_Status,Hometown,Unit,Decision_skill_possess,Time_of_service,Time_since_promotion,growth_rate,Travel_Rate,Post_Level,Pay_Scale,Compensation_and_Benefits,Work_Life_balance,Attrition_rate
0,EID_23371,F,42.0,4,Married,Franklin,IT,Conceptual,4.0,4,33,1,1,7.0,type2,3.0,0.1841
1,EID_18000,M,24.0,3,Single,Springfield,Logistics,Analytical,5.0,4,36,0,3,6.0,type2,4.0,0.0670
2,EID_3891,F,58.0,3,Married,Clinton,Quality,Conceptual,27.0,3,51,0,2,8.0,type2,1.0,0.0851
3,EID_17492,F,26.0,3,Single,Lebanon,Human Resource Management,Behavioral,4.0,3,56,1,3,8.0,type2,1.0,0.0668
4,EID_22534,F,31.0,1,Married,Springfield,Logistics,Conceptual,5.0,4,62,1,3,2.0,type3,3.0,0.1827
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,EID_16328,F,23.0,5,Married,Franklin,Operarions,Behavioral,4.0,1,20,1,2,6.0,type3,3.0,0.7839
6996,EID_8387,F,44.0,1,Married,Lebanon,R&D,Analytical,16.0,2,59,0,2,4.0,type3,3.0,0.1055
6997,EID_8077,F,49.0,3,Single,Springfield,IT,Directive,10.0,1,59,1,2,3.0,type2,4.0,0.7847
6998,EID_19597,F,47.0,3,Married,Washington,Sales,Behavioral,24.0,1,21,1,5,8.0,type0,2.0,0.4162


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7000 entries, 0 to 6999
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Employee_ID                7000 non-null   object 
 1   Gender                     7000 non-null   object 
 2   Age                        6588 non-null   float64
 3   Education_Level            7000 non-null   int64  
 4   Relationship_Status        7000 non-null   object 
 5   Hometown                   7000 non-null   object 
 6   Unit                       7000 non-null   object 
 7   Decision_skill_possess     7000 non-null   object 
 8   Time_of_service            6856 non-null   float64
 9   Time_since_promotion       7000 non-null   int64  
 10  growth_rate                7000 non-null   int64  
 11  Travel_Rate                7000 non-null   int64  
 12  Post_Level                 7000 non-null   int64  
 13  Pay_Scale                  6991 non-null   float

In [11]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
Age,6588.0,39.622799,13.606920,19.0,27.0000,37.00000,52.000,65.0000
Education_Level,7000.0,3.187857,1.065102,1.0,3.0000,3.00000,4.000,5.0000
Time_of_service,6856.0,13.385064,10.364188,0.0,5.0000,10.00000,21.000,43.0000
Time_since_promotion,7000.0,2.367143,1.149395,0.0,1.0000,2.00000,3.000,4.0000
growth_rate,7000.0,47.064286,15.761406,20.0,33.0000,47.00000,61.000,74.0000
Travel_Rate,7000.0,0.817857,0.648205,0.0,0.0000,1.00000,1.000,2.0000
Post_Level,7000.0,2.798000,1.163721,1.0,2.0000,3.00000,3.000,5.0000
Pay_Scale,6991.0,6.006294,2.058435,1.0,5.0000,6.00000,8.000,10.0000
Work_Life_balance,6989.0,2.387895,1.122786,1.0,1.0000,2.00000,3.000,5.0000
Attrition_rate,7000.0,0.189376,0.185753,0.0,0.0704,0.14265,0.235,0.9959


In [78]:
data = data.groupby(by="Hometown").mean(numeric_only=True)
data.reset_index()

,Hometown,Age,Education_Level,Time_of_service,Time_since_promotion,growth_rate,Travel_Rate,Post_Level,Pay_Scale,Work_Life_balance,Attrition_rate
0,Clinton,39.075901,3.166372,12.805755,2.315044,47.334513,0.858407,2.775221,6.044405,2.463717,0.188464
1,Franklin,40.093023,3.187131,13.750167,2.369665,47.018385,0.826001,2.774787,6.009211,2.400394,0.187901
2,Lebanon,40.050026,3.215459,13.761128,2.363768,46.544444,0.800000,2.800966,5.987911,2.372037,0.187019
3,Springfield,39.092683,3.168203,13.079505,2.376152,47.099654,0.822005,2.813364,6.064553,2.392610,0.194084
4,Washington,39.290508,3.179024,12.954755,2.382459,47.906872,0.812839,2.811935,5.925792,2.354167,0.188895
